In [ ]:
import pandas as pd

# Load CSV file into a DataFrame
# Replace 'your_file.csv' with the actual path to your CSV file
df = pd.read_csv('C:/Users/dyros/Desktop/dummy_ws/data/AApose10.csv')

# Display the first few rows of the DataFrame
df


In [ ]:
matching_indices = df[df['field.trigger_flag'] == 1].index
matching_indices

In [ ]:

columns_to_drop = df.columns[0:7]
print(df.columns)
cols_to_check = ['field.pose_array.poses0.position.x', 'field.pose_array.poses0.position.y',"field.pose_array.poses0.position.z"]
df = df.drop(columns=columns_to_drop)
dup_mask = df.duplicated(subset=cols_to_check, keep=False)

# 3) Keep only rows that are unique in the specified columns
df = df[~dup_mask].reset_index(drop=True)
# Step 3: Check remaining columns

# columns_to_drop = df.columns[[1, 3]]  # e.g., ['age', 'email']
df
# # Step 2: Drop those columns
# df = df.drop(columns=columns_to_drop)

In [ ]:
matching_indices = df[df['field.trigger_flag'] == 1].index
matching_indices

In [ ]:
import pandas as pd
from typing import Union, List
import numpy as np

def apply_reference_values(
    df: pd.DataFrame,
    row_start: int,
    row_end: int,
    col_indices: Union[slice, List[int], tuple],
    noise_std: float = 1.0
) -> pd.DataFrame:
    """
    Overwrite values in specified column range for rows from row_start to row_end - 1
    using the values from row_start as reference.

    Parameters:
    - df (pd.DataFrame): the DataFrame to modify
    - row_start (int): start index (inclusive)
    - row_end (int): end index (exclusive)
    - col_indices (slice or list or tuple): column indices to be modified

    Returns:
    - pd.DataFrame: modified DataFrame (copy)
    """
    df_copy = df.copy()

    # Convert column indices (int or slice) to actual column names
    if isinstance(col_indices, slice):
        cols = df.columns[col_indices]
    elif isinstance(col_indices, (list, tuple)):
        cols = df.columns[list(col_indices)]
    else:
        raise TypeError("col_indices must be a slice, list, or tuple of integers")

    # Get reference values from the start row
    reference_values = df.loc[row_start, cols]

    # Overwrite target range rows with reference values
    noise = np.random.normal(loc=0.0, scale=noise_std, size=df_copy.loc[row_start:row_end - 1, cols].shape)
    df_copy.loc[row_start:row_end - 1, cols] = reference_values.values
    df_copy.loc[row_start:row_end - 1, cols] = df_copy.loc[row_start:row_end - 1, cols] + noise

    return df_copy


In [ ]:
std_1 = 0.5  # for -9:-5
std_2 = 1  # for -5:-1

new_df = df.copy()

for i in range(len(matching_indices) // 2):
    row_start = matching_indices[2 * i]
    row_end = matching_indices[2 * i + 1]
    
    # First range: columns -9:-5 with std_1
    new_df = apply_reference_values(
        new_df,
        row_start=row_start,
        row_end=row_end,
        col_indices=slice(-12, -8),
        noise_std=std_1
    )
    
    # Second range: columns -5:-1 with std_2
    new_df = apply_reference_values(
        new_df,
        row_start=row_start,
        row_end=row_end,
        col_indices=slice(-8, -4),
        noise_std=std_2
    )


In [ ]:
new_df.to_csv('modified_output_AA10.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd
import glob
import os

# Step 1: 파일 경로 패턴 설정 (예: 'data/file1.csv' ~ 'data/file8.csv')
file_paths = [
    'modified_output_1.csv',
    'modified_output_2.csv',
    'modified_output_3.csv',
    'modified_output_4.csv',
    'modified_output_5.csv',
    'modified_output_6.csv',
    'modified_output_7.csv',
    'modified_output_9.csv',
    'modified_output_10.csv',
    'modified_output_11.csv',
    'modified_output_12.csv',
    'modified_output_13.csv',
    'modified_output_pose1.csv',
    'modified_output_pose2.csv',
    'modified_output_pose3.csv',
    'modified_output_pose4.csv',
    'modified_output_pose5.csv'
]

# Step 2: 각 CSV 파일을 읽어 DataFrame 리스트로 저장
df_list = [pd.read_csv(path) for path in file_paths]

# Step 3: 리스트의 모든 DataFrame을 하나로 연결
modify_df = pd.concat(df_list, ignore_index=True)
modify_df = modify_df.dropna()
# Step 4: 결과 확인 또는 저장

cols_to_check = ['field.pose_array.poses0.position.x', 'field.pose_array.poses0.position.y',"field.pose_array.poses0.position.z"]
dup_mask = modify_df.duplicated(subset=cols_to_check, keep=False)

# 3) Keep only rows that are unique in the specified columns
modify_df = modify_df[~dup_mask].reset_index(drop=True)


In [ ]:
def compute_euler_with_fixed_ref(q_ref: np.ndarray,
                                 q_targets: np.ndarray):
    """
    Compute roll, pitch, yaw of each q_target relative to a fixed q_ref.
    
    Parameters:
      q_ref      : array of shape (4,), [w, x, y, z]
      q_targets  : array of shape (N,4), [w, x, y, z] per row
    
    Returns:
      roll       : array of shape (N,), rotation about X-axis (radians)
      pitch      : array of shape (N,), rotation about Y-axis (radians)
      yaw        : array of shape (N,), rotation about Z-axis (radians)
    """
    # 1) Normalize reference quaternion
    q1 = q_ref.astype(float)
    q1 /= np.linalg.norm(q1)
    
    # 2) Normalize all target quaternions
    norms = np.linalg.norm(q_targets, axis=1, keepdims=True)
    q2 = q_targets.astype(float) / norms

    # 3) Conjugate of q_ref: (w, -x, -y, -z)
    q1_conj = q1.copy()
    q1_conj[1:] *= -1  # invert vector part

    # 4) Broadcast q1_conj to shape (N,4) for multiplication
    w1, x1, y1, z1 = q1_conj
    w2, x2, y2, z2 = q2.T

    # quaternion multiplication q_rel = q1_conj * q2
    w_rel =  w1*w2 - x1*x2 - y1*y2 - z1*z2
    x_rel =  w1*x2 + x1*w2 + y1*z2 - z1*y2
    y_rel =  w1*y2 - x1*z2 + y1*w2 + z1*x2
    z_rel =  w1*z2 + x1*y2 - y1*x2 + z1*w2

    # 5) Convert to Euler angles
    roll  = np.arctan2(2*(w_rel*x_rel + y_rel*z_rel),
                       1 - 2*(x_rel**2 + y_rel**2))
    pitch = np.arcsin(2*(w_rel*y_rel - z_rel*x_rel))
    yaw   = np.arctan2(2*(w_rel*z_rel + x_rel*y_rel),
                       1 - 2*(y_rel**2 + z_rel**2))

    return roll, pitch, yaw

def add_euler_columns(df: 'pd.DataFrame',
                      q_ref: np.ndarray,
                      target_cols: list,
                      col_names: list = ['roll', 'pitch', 'yaw']
                     ) -> 'pd.DataFrame':
    """
    Compute roll, pitch, yaw angles for each quaternion in df[target_cols]
    relative to a fixed reference quaternion q_ref, and assign them as new columns.

    Parameters:
      df         : pd.DataFrame containing quaternion data
      q_ref      : np.ndarray of shape (4,), [w, x, y, z]
      target_cols: list of 4 column names in df representing [w, x, y, z]
      col_names  : list of 3 names for the new Euler angle columns

    Returns:
      df_out     : a new DataFrame with the added Euler columns
    """
    # 1) Extract target quaternions as NumPy array
    q_targets = df[target_cols].to_numpy()  # shape (N,4)

    # 2) Compute Euler angles relative to q_ref
    roll_vals, pitch_vals, yaw_vals = compute_euler_with_fixed_ref(q_ref, q_targets)

    # 3) Assign to new columns in a copy of the DataFrame
    df_out = df.copy()
    df_out[col_names[0]] = roll_vals
    df_out[col_names[1]] = pitch_vals
    df_out[col_names[2]] = yaw_vals

    return df_out
# ===== Usage Example =====

# suppose df is your DataFrame with these columns:
# ['ref_w','ref_x','ref_y','ref_z','tgt_w','tgt_x','tgt_y','tgt_z']
import pandas as pd
df = modify_df

# extract as numpy arrays
# Suppose df is your DataFrame, q_ref is defined
target_cols = ['field.pose_array.poses0.orientation.w',
               'field.pose_array.poses0.orientation.x',
               'field.pose_array.poses0.orientation.y',
               'field.pose_array.poses0.orientation.z']
q_ref = np.array([0, 0, -np.sqrt(0.5), np.sqrt(0.5)])

df_with_euler = add_euler_columns(df, q_ref, target_cols,col_names=['field.angles8', 'field.angles9', 'field.angles10'])
print(df_with_euler[['field.angles8', 'field.angles9', 'field.angles10']].head())





In [ ]:
import pandas as pd
import glob
import os

# Step 1: 파일 경로 패턴 설정 (예: 'data/file1.csv' ~ 'data/file8.csv')
file_paths = [
    'modified_output_AA1.csv',
    'modified_output_AA2.csv',
    'modified_output_AA3.csv',
    'modified_output_AA4.csv',
    'modified_output_AA5.csv',
    'modified_output_AA6.csv',
    'modified_output_AA7.csv',
    'modified_output_AA8.csv',
    'modified_output_AA9.csv',
    'modified_output_AA10.csv'
]

# Step 2: 각 CSV 파일을 읽어 DataFrame 리스트로 저장
df_list = [pd.read_csv(path) for path in file_paths]

# Step 3: 리스트의 모든 DataFrame을 하나로 연결
merged_df = pd.concat(df_list, ignore_index=True)
merged_df = merged_df.dropna()
# Step 4: 결과 확인 또는 저장
print(merged_df.shape)   # (총 행 수, 열 수)
print(merged_df.head())  # 상위 5개 행 미리보기




In [ ]:
cols_to_check = ['field.pose_array.poses0.position.x', 'field.pose_array.poses0.position.y',"field.pose_array.poses0.position.z"]
dup_mask = merged_df.duplicated(subset=cols_to_check, keep=False)

# 3) Keep only rows that are unique in the specified columns
merged_df = merged_df[~dup_mask].reset_index(drop=True)

# df_merged = pd.concat([df_with_euler, merged_df], axis=0, ignore_index=True)
# df_merged

In [ ]:
import re

# List of keywords to remove
keywords = ['orientation', 'poses0', 'poses21', 'poses22', 'poses23', 'poses24', 'poses25']
pattern = '|'.join(keywords)

# 1) Find columns containing any of the keywords
# mask_any_keyword = df_merged.columns.str.contains(pattern)
mask_any_keyword = merged_df.columns.str.contains(pattern)

# 2) Identify columns that contain both 'orientation' and 'poses0'
# mask_both = (
#     df_merged.columns.str.contains('orientation') &
#     df_merged.columns.str.contains('poses0')
# )

mask_both = (
    merged_df.columns.str.contains('orientation') &
    merged_df.columns.str.contains('poses0')
)

# 3) Build final mask:
#    - Keep if NOT containing any keyword, OR
#    - Keep if containing both 'orientation' and 'poses0'
final_mask = ~mask_any_keyword 

# 4) Apply mask to DataFrame
# df_merged = df_merged.loc[:, final_mask]
df_merged = merged_df.loc[:, final_mask]


In [ ]:
cols = df_merged.columns.tolist()
new_cols = (
    cols[:60] +
    cols[68:71] +
    cols[60:68] +    
    [cols[71]]
)
df_merged = df_merged[new_cols]

In [ ]:
df_merged = df_merged[df_merged['field.angles1'] >= -25]
df_merged = df_merged[df_merged['field.angles2'] >= -25]
df_merged = df_merged[df_merged['field.angles3'] >= -25]

df_merged = df_merged[df_merged['field.angles1'] <= 25]
df_merged = df_merged[df_merged['field.angles2'] <= 25]
df_merged = df_merged[df_merged['field.angles3'] <= 25]

In [ ]:
import torch

df_merged.columns
X = df_merged.iloc[:, :-9].values         
y = df_merged.iloc[:, -8:-5].values          


X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

X.shape


In [ ]:
import torch
import torch.nn.functional as F

def remove_rows_with_outliers(
    y_tensor: torch.Tensor,
    X: torch.Tensor,
    window_size: int = 31,
    abs_diff_thresh: float = 10.0
):
    """
    Detect outliers in each column of y_tensor by
      |y[t,c] - local_window_mean[t,c]| > abs_diff_thresh,
    then remove any row t containing an outlier in ANY column,
    from BOTH y_tensor and X.

    Parameters:
      y_tensor        : torch.Tensor of shape [T, C]
      X               : torch.Tensor of shape [T, ...] (same first dim T)
      window_size     : odd int, size of the rolling-mean window
      abs_diff_thresh : float, absolute difference threshold

    Returns:
      clean_y         : torch.Tensor [T - num_removed, C]
      clean_X         : torch.Tensor [T - num_removed, ...]
      removed_indices : 1D LongTensor of removed row indices
    """
    assert y_tensor.dim() == 2 and X.dim() >= 2
    T, C = y_tensor.shape
    assert X.size(0) == T, "X must have same number of rows as y_tensor"
    assert window_size % 2 == 1, "window_size must be odd"

    pad = window_size // 2
    kernel = torch.ones(window_size, device=y_tensor.device) / window_size
    kernel = kernel.view(1, 1, -1)

    # 마스크 초기화
    remove_mask = torch.zeros(T, dtype=torch.bool, device=y_tensor.device)

    # 각 컬럼마다 rolling mean 계산 후 이상치 마킹
    for c in range(C):
        ts = y_tensor[:, c].view(1,1,T)
        ts_padded = F.pad(ts, (pad, pad), mode='reflect')
        rolling_mean = F.conv1d(ts_padded, kernel).view(-1)
        diff = (y_tensor[:, c] - rolling_mean).abs()
        remove_mask |= (diff > abs_diff_thresh)

    removed_indices = remove_mask.nonzero(as_tuple=True)[0]

    # 행 제거
    clean_y = y_tensor[~remove_mask]
    clean_X = X[~remove_mask]

    return clean_y, clean_X, removed_indices


# ===== Usage Example =====
# y_tensor: [T, 3], X: [T, D]
clean_y, clean_X, removed_idx = remove_rows_with_outliers(
    y_tensor,
    X_tensor,
    window_size=401,
    abs_diff_thresh=5.0
)

print("Removed row indices:", removed_idx.tolist())
print("Original shapes:", y_tensor.shape, X.shape)
print("Cleaned shapes: ", clean_y.shape, clean_X.shape)


In [ ]:
torch.save((clean_X, clean_y), 'dataset.pt')

In [ ]:
import pandas as pd
import torch
import joblib
import numpy as np

# Step 1: 데이터 불러오기 및 중복 제거
test_df = pd.read_csv("modified_output_AA_test.csv")
test_df = test_df.dropna()
cols_to_check = ['field.pose_array.poses0.position.x', 'field.pose_array.poses0.position.y',"field.pose_array.poses0.position.z"]
dup_mask = test_df.duplicated(subset=cols_to_check, keep=False)

test_df = add_euler_columns(test_df, q_ref, target_cols,col_names=['field.angles8', 'field.angles9', 'field.angles10'])

# 3) Keep only rows that are unique in the specified columns
test_df = test_df[~dup_mask].reset_index(drop=True)
test_df = test_df[new_cols]

import re

# List of keywords to remove
keywords = ['orientation', 'poses0', 'poses21', 'poses22', 'poses23', 'poses24', 'poses25']
pattern = '|'.join(keywords)

# 1) Find columns containing any of the keywords
mask_any_keyword = test_df.columns.str.contains(pattern)

# 2) Identify columns that contain both 'orientation' and 'poses0'
mask_both = (
    test_df.columns.str.contains('orientation') &
    test_df.columns.str.contains('poses0')
)

# 3) Build final mask:
#    - Keep if NOT containing any keyword, OR
#    - Keep if containing both 'orientation' and 'poses0'
final_mask = ~mask_any_keyword 

# 4) Apply mask to DataFrame
test_df = test_df.loc[:, final_mask]
test_df = test_df[new_cols]

# Step 2: X, y 분리
X_test = torch.tensor(test_df.iloc[:, :-9].values, dtype=torch.float32)
y = test_df.iloc[:, -8:-5].values.astype(float)

# Step 3: 값이 160보다 큰 경우 부호 반전
# y = np.where(y_raw > 160, -y_raw, y_raw)

# Step 4: 저장된 스케일러 불러오기
# scaler = joblib.load('y_scaler.pkl')

# Step 5: y에 스케일러 적용
# y_scaled = scaler.transform(y)

# Step 6: 텐서로 변환 후 저장
y_test = torch.tensor(y, dtype=torch.float32)
torch.save((X_test, y_test), 'test.pt')


In [ ]:
nan_locations = test_df.isna()

# Get list of positions (row index, column name) where NaN exists
nan_positions = [(row_idx, col) for row_idx, row in nan_locations.iterrows() for col, is_nan in row.items() if is_nan]

# Print the results
if nan_positions:
    print(f'Total NaN count: {len(nan_positions)}')
    print('NaN locations:')
    for pos in nan_positions:
        print(f'Row index: {pos[0]}, Column: {pos[1]}')
else:
    print('There are no NaN values in the DataFrame.')

In [ ]:
import matplotlib.pyplot as plt
idx=1
# Plot histogram as normalized (probability density)
plt.hist(y_tensor[:, idx], bins=20, edgecolor='black', alpha=0.6, density=True, label='merged_df')
plt.hist(y_test[:, idx], bins=20, edgecolor='black', alpha=0.6, density=True, label='test_df')

plt.title('Normalized Histogram of Angle')
plt.xlabel('Age')
plt.ylabel('Density')  # not Frequency, since we're using density=True
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
plt.subplot(2,1,1)
plt.plot(y_tensor[:,0])
plt.subplot(2,1,2)
plt.plot(clean_y[:,0])

plt.show()


In [ ]:
import numpy as np

# Replace with your actual .npz path
npz_path = 'model_weights.npz'

try:
    data = np.load(npz_path)
    print(f"Loaded .npz file: {npz_path}")
    print(f"Found {len(data.files)} arrays:\n")

    for key in data.files:
        arr = data[key]
        shape = arr.shape
        ndim = arr.ndim

        # Heuristic classification:
        if ndim == 2:
            # 2D weight: likely Linear.weight
            # shape = (out_features, in_features)
            print(f"  - {key}: shape={shape}, dtype={arr.dtype} => likely Linear.weight")
        elif ndim == 1:
            # 1D: could be LayerNorm.weight or LayerNorm.bias or Linear.bias.
            # 추가적으로 이름으로 유추:
            if key.endswith('.weight'):
                # 1D weight: if preceding layer is LayerNorm(prev_dim), then this is LayerNorm.weight.
                print(f"  - {key}: shape={shape}, dtype={arr.dtype} => 1D weight: likely LayerNorm.weight")
            elif key.endswith('.bias'):
                # 1D bias: could be LayerNorm.bias or Linear.bias.
                # 이름만으로 구분이 어려우므로, 앞의 인덱스와 shape를 보고 추론 필요.
                # 예: ffn.0.bias shape=(182,)인데, ffn.0.weight도 (182,)이므로 LayerNorm.bias.
                #     ffn.1.bias shape=(512,)인데 weight shape (512,182)이면 Linear.bias.
                # 간단히 두 가지 모두 가능하다고 표시:
                print(f"  - {key}: shape={shape}, dtype={arr.dtype} => 1D bias: LayerNorm.bias or Linear.bias")
            else:
                print(f"  - {key}: shape={shape}, dtype={arr.dtype} => 1D array: unknown role")
        else:
            print(f"  - {key}: shape={shape}, dtype={arr.dtype} => unexpected ndim={ndim}")

    # 추가로, 1D bias가 Linear.bias인지 LayerNorm.bias인지 정확히 구분하려면:
    print("\nAdditional check for 1D bias vs LayerNorm.bias:")
    for key in data.files:
        if key.endswith('.bias') and data[key].ndim == 1:
            idx = key.split('.')[1]  # 'ffn.<idx>.bias'
            weight_key = f"ffn.{idx}.weight"
            if weight_key in data.files:
                w = data[weight_key]
                if w.ndim == 1 and w.shape == data[key].shape:
                    # weight도 1D, bias shape 같음 → LayerNorm.bias
                    role = "LayerNorm.bias (matched 1D weight)"
                elif w.ndim == 2 and w.shape[0] == data[key].shape[0]:
                    # weight is 2D with out_features equal bias length → Linear.bias
                    role = "Linear.bias (matched 2D weight out_features)"
                else:
                    role = "Unknown bias role"
            else:
                role = "No matching weight key; unknown"
            print(f"  - {key}: shape={data[key].shape} => {role}")

except FileNotFoundError:
    print(f"File not found: {npz_path}. Please ensure the file exists or update the path.")
except Exception as e:
    print(f"An error occurred while loading or processing the npz file: {e}")


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
from model import  Skeleton2Mesh
# 2) Load test data
X_test, y_test = torch.load('test.pt',weights_only=False)

# 1) Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3) Hyperparameters / model config
num_joints    = 20
num_bones     = 19
gsd_dim       = 100
mlp_hidden    = [256, 256]
pe_freqs_bone = 5
pe_freqs_order= 2

# 4) Instantiate model and load checkpoint
model = Skeleton2Mesh(
    num_joints=num_joints,
    num_bones=num_bones,
    gsd_dim=gsd_dim,
    mlp_hidden=mlp_hidden,
    pe_freqs_bone=pe_freqs_bone,
    pe_freqs_order=pe_freqs_order
).to(device)

# Path to saved checkpoint
checkpoint_path = "checkpoint_euler_epoch1000_ver2.pth"  # 실제 파일명으로 변경
# checkpoint_path = "checkpoint_epoch900.pth"  # 실제 파일명으로 변경

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # set to evaluation mode

# If optimizer state or epoch 등 추가 정보가 필요하면
# epoch_loaded = checkpoint.get('epoch')
# loss_loaded = checkpoint.get('loss')

# 5) Prepare test data
#    Assume you have NumPy arrays:
#      skeletons_test_np: shape [N_test, num_joints*3]
#      angles_test_np:    shape [N_test, 8]
#    These must be prepared beforehand.
import numpy as np
# Example placeholders; 실제 데이터를 여기에 할당하세요.
# skeletons_test_np = np.load("skeletons_test.npy")  # [N_test, 63]
# angles_test_np    = np.load("angles_test.npy")     # [N_test, 8]

# Convert to torch.Tensor

# Create DataLoader
batch_size = 64  # 필요에 따라 변경
test_dataset = TensorDataset(X_test, y_test)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# 6) Define evaluation metrics
mse_loss_fn = nn.MSELoss(reduction='mean')
mae_loss_fn = nn.L1Loss(reduction='mean')

# Optional: per-dimension metrics
num_outputs = y_test.shape[1]  # e.g. 8

# Containers for aggregated results
total_mse = 0.0
total_mae = 0.0
num_samples = 0

# If you want per-output errors, accumulate sums:
sum_sq_errors = torch.zeros(num_outputs, device=device)
sum_abs_errors = torch.zeros(num_outputs, device=device)

# 7) Evaluation loop
with torch.no_grad():
    for skeletons_flat_batch, angles_batch in test_loader:
        skeletons_flat_batch = skeletons_flat_batch.to(device)  # [B, num_joints*3]
        angles_batch         = angles_batch.to(device)         # [B, 8]

        # Forward pass
        preds = model(skeletons_flat_batch)  # [B, 8]

        # Compute batch losses
        batch_mse = mse_loss_fn(preds, angles_batch)  # scalar
        batch_mae = mae_loss_fn(preds, angles_batch)  # scalar

        # Accumulate weighted by batch size
        B = skeletons_flat_batch.size(0)
        total_mse += batch_mse.item() * B
        total_mae += batch_mae.item() * B
        num_samples += B

        # Per-output accumulation
        # Compute squared error and abs error per sample per dimension
        diff = preds - angles_batch  # [B, 8]
        sum_sq_errors += torch.sum(diff * diff, dim=0)    # [8]
        sum_abs_errors += torch.sum(torch.abs(diff), dim=0)  # [8]

# 8) Final metrics
mean_mse = total_mse / num_samples
mean_mae = total_mae / num_samples

# Per-output RMSE and MAE
rmse_per_output = torch.sqrt(sum_sq_errors / num_samples)  # [8]
mae_per_output  = sum_abs_errors / num_samples            # [8]

print(f"Test MSE (mean over all outputs): {mean_mse:.6f}")
print(f"Test MAE (mean over all outputs): {mean_mae:.6f}")
print("Per-output RMSE:", rmse_per_output.cpu().numpy())
print("Per-output MAE: ", mae_per_output.cpu().numpy())

In [ ]:
# 원하는 인덱스를 지정
index = 791 # 원하는 인덱스로 변경 가능

# 1) 입력 데이터 추출 및 모델로 예측
x_sample = X_test[index].unsqueeze(0).to(device)  # [1, input_dim]
y_true   = y_test[index].cpu().numpy()            # [output_dim]

model.eval()
with torch.no_grad():
    y_pred = model(x_sample).cpu().numpy().flatten()  # [output_dim]

# 2) 결과 출력
print(f"Index: {index}")
print("Predicted:", y_pred)
print("Ground Truth:", y_true)

# 3) Optional: 차이 출력
print("Absolute Error:", np.abs(y_pred - y_true))


In [ ]:
# Ensure model is in eval mode
model.eval()

# 1) 예측값과 true값을 모두 저장할 리스트 초기화
all_preds = []
all_targets = []

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        preds = model(x_batch)

        all_preds.append(preds.cpu())
        all_targets.append(y_batch.cpu())

# 2) 모든 예측값과 실제값을 연결 (concat)
all_preds_tensor = torch.cat(all_preds, dim=0)       # shape [N, output_dim]
all_targets_tensor = torch.cat(all_targets, dim=0)   # shape [N, output_dim]

# 3) 샘플별 MSE 계산
mse_per_sample = torch.mean((all_preds_tensor - all_targets_tensor) ** 2, dim=1)  # shape [N]

# 4) 최대 MSE 인덱스 탐색
max_mse_value, max_mse_index = torch.max(mse_per_sample, dim=0)

# 5) 해당 인덱스의 예측값과 실제값
y_pred_max = all_preds_tensor[max_mse_index].numpy()
y_true_max = all_targets_tensor[max_mse_index].numpy()

# 6) 출력
print(f"Index with Maximum MSE: {max_mse_index.item()}")
print(f"Maximum MSE value: {max_mse_value.item():.6f}")
print("Predicted   :", y_pred_max)
print("Ground Truth:", y_true_max)
print("Abs Error   :", np.abs(y_pred_max - y_true_max))


In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter

# ---------------------------
# 1. EMA 필터 함수
# ---------------------------
def apply_ema(tensor: torch.Tensor, alpha: float = 0.1) -> torch.Tensor:
    """
    Apply EMA (Exponential Moving Average) to each column of a 2D tensor.
    """
    df = pd.DataFrame(tensor.numpy())
    ema_df = df.ewm(alpha=alpha, adjust=False).mean()
    return torch.tensor(ema_df.values, dtype=tensor.dtype)

# ---------------------------
# 2. Butterworth 필터 함수
# ---------------------------
def apply_butterworth(tensor: torch.Tensor, cutoff: float, fs: float, order: int = 2) -> torch.Tensor:
    """
    Apply Butterworth low-pass filter to each column of a 2D tensor.
    """
    b, a = butter(N=order, Wn=cutoff / (fs / 2), btype='low')
    tensor_np = tensor.numpy()
    filtered_np = np.zeros_like(tensor_np)
    
    for i in range(tensor_np.shape[1]):
        filtered_np[:, i] = lfilter(b, a, tensor_np[:, i])
    
    return torch.tensor(filtered_np, dtype=tensor.dtype)

# ---------------------------
# 3. 필터 적용
# ---------------------------
ema_preds_tensor     = apply_ema(all_preds_tensor, alpha=0.5)
butter_preds_tensor  = apply_butterworth(all_preds_tensor, cutoff=10.0, fs=60.0, order=2)
all_filter_preds_tensor     = apply_ema(butter_preds_tensor, alpha=0.5)
# butter_preds_tensor = ema_preds_tensor

# ---------------------------
# 4. 시각화
# ---------------------------
plt.figure(figsize=(10,12))

plt.subplot(4,1,1)
plt.plot(mse_per_sample.numpy(), marker='o', linestyle='-', markersize=2)
plt.title('MSE per Sample Index')
plt.xlabel('Sample Index')
plt.ylabel('MSE')
plt.grid(True)
plt.tight_layout()

mse_ema_sample = torch.mean((ema_preds_tensor - all_targets_tensor) ** 2, dim=1)  # shape [N]

plt.subplot(5,1,2)
plt.plot(mse_ema_sample.numpy(), marker='o', linestyle='-', markersize=2)

prev_butter = butter_preds_tensor[0,:]
prev_ema = ema_preds_tensor[0,:]
prev_all = all_filter_preds_tensor[0,:]

for i in range(1,len(butter_preds_tensor)):
    butter_preds_tensor[i,:] = np.clip(butter_preds_tensor[i,:],-0.15 + prev_butter,0.15+ prev_butter)
    ema_preds_tensor[i,:] = np.clip(ema_preds_tensor[i,:],-0.15 + prev_ema,0.15+ prev_ema)
    all_filter_preds_tensor[i,:] = np.clip(all_filter_preds_tensor[i,:],-0.15 + prev_all,0.15+ prev_all)
    prev_butter = butter_preds_tensor[i,:]
    prev_ema = ema_preds_tensor[i,:]
    prev_all = all_filter_preds_tensor[i,:]

for i in range(2,5):
    plt.subplot(5,1,i+1)
    plt.plot(all_preds_tensor[:, i-2], label="pred", alpha=0.4)
    plt.plot(ema_preds_tensor[:, i-2], label="pred (EMA)", linewidth=1.8)
    plt.plot(butter_preds_tensor[:, i-2], label="pred (Butter)", linewidth=1.8)
    plt.plot(all_filter_preds_tensor[:, i-2], label="pred (all)", linewidth=1.8)
    plt.plot(all_targets_tensor[:, i-2], label="target", linestyle='--')
    plt.legend()

plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(mse_per_sample.numpy(), bins=50, edgecolor='black')
plt.title('Histogram of MSE per Sample')
plt.xlabel('MSE Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
for i in range(1,11):
# Path to saved checkpoint
    checkpoint_path = f"checkpoint_euler_epoch{i*100}.pth"  # 실제 파일명으로 변경
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    print(checkpoint['loss'])


In [ ]:
X_tensor, y_tensor = torch.load('dataset.pt')  # X: [N,182], y: [N]
X_tensor.shape